# Tools and Chains

Inspiration from: https://www.youtube.com/watch?v=hI2BY7yl_Ac

Side note: It might be a fun project to see if we can extract information directly from the youtube video and finalize that output into code

In [2]:
import os
from langchain.prompts  import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

In [ ]:
# Make sure you have setup your environmental variables beforehand
os.environ["OPENAI_API_KEY"]
os.environ["OPENAI_ORGANIZATION"]

### Basic LLM Chain: Testing fact extraction

Start by loading in model of choice

In [22]:
llm_openai = OpenAI(model_name="gpt-4-0613", temperature=0, max_tokens=512)

/opt/homebrew/Caskroom/miniforge/base/envs/open_ai/lib/python3.9/site-packages/langchain/llms/openai.py:200: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/open_ai/lib/python3.9/site-packages/langchain/llms/openai.py:785: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


Let's feed an article to extract facts from, this is the free completely unstructured text

In [6]:
article = """
Paris Saint-Germain soccer superstar Neymar has reportedly agreed a two-year deal to join Saudi Arabian club Al-Hilal.

France’s leading sports daily L’Equipe said on its website Sunday that the PSG and Brazil forward will receive a total of 160 million euros ($175 million) over two seasons. No details were given as to how much the transfer fee would be for Neymar, who has also been linked with a return to Barcelona.

The Associated Press has asked defending French champion PSG to confirm the deal. The request was not immediately answered.

Neymar missed PSG’s season-opening 0-0 draw against Lorient in the French league on Saturday after training alone on Friday, with the club saying it was because he was recovering from a viral infection. But Neymar is heading for a PSG exit and could leave during the summer transfer window.

The 31-year-old Brazilian joined PSG from Barcelona for a world-record fee of 222 million euros (now $244 million) six years ago, the same year Kylian Mbappe joined from Monaco in a deal worth 180 million euros.

PSG had already received a world-record $332 million bid from Al-Hilal for Mbappe, who is in the last year of his contract and has been mired in a transfer standoff. Mbappe, who reportedly refused to meet with representatives from the Saudi club when they were in Paris last month, has been strongly linked with a move to Real Madrid next season on a free transfer.

Tensions between Mbappe and PSG eased a little on Sunday after he was allowed to return to training following “constructive and positive talks” between the two parties.

Mbappe sat out Saturday’s draw with Lorient after being left out of training. Midfielder Marco Verratti was also not selected by new coach Luis Enrique, and has been linked with a move to the Saudi league after joining PSG in 2012.

"""

Article is pretty long, around 1,800 characters

In [7]:
len(article)

1823

Example of general PromptTemplate usage

In [19]:
instructions = "Extract the key facts about this piece of text. Don't include opinions. Give each fact a number and keep short sentences:\n {text_input}"

fact_extraction_prompt = PromptTemplate(input_variables=["text_input"],
                                        template=instructions)
print(fact_extraction_prompt.format(text_input=article))

Extract the key facts about this piece of text. Don't include opinions. Give each fact a number and keep short sentences:
 
Paris Saint-Germain soccer superstar Neymar has reportedly agreed a two-year deal to join Saudi Arabian club Al-Hilal.

France’s leading sports daily L’Equipe said on its website Sunday that the PSG and Brazil forward will receive a total of 160 million euros ($175 million) over two seasons. No details were given as to how much the transfer fee would be for Neymar, who has also been linked with a return to Barcelona.

The Associated Press has asked defending French champion PSG to confirm the deal. The request was not immediately answered.

Neymar missed PSG’s season-opening 0-0 draw against Lorient in the French league on Saturday after training alone on Friday, with the club saying it was because he was recovering from a viral infection. But Neymar is heading for a PSG exit and could leave during the summer transfer window.

The 31-year-old Brazilian joined PSG 

Now we have a template with the article we are interested on in the template. Let's go ahead to build the chain and input it

Notice how build the chain without the actual artcile input in the prompt, but just the prompt template with the instructions

In [23]:
fact_extraction_chain = LLMChain(llm=llm_openai, prompt=fact_extraction_prompt)

Now we run the extraction chain with the actual article in the input "text_input" that we indicated in the prompt template

In [24]:
print(fact_extraction_chain.run(article))

1. Paris Saint-Germain (PSG) soccer player Neymar has reportedly agreed to a two-year deal with Saudi Arabian club Al-Hilal.
2. Neymar will receive a total of 160 million euros ($175 million) over two seasons.
3. No details were given about the transfer fee for Neymar.
4. Neymar has been linked with a return to Barcelona.
5. The Associated Press has asked PSG to confirm the deal.
6. Neymar missed PSG’s season-opening 0-0 draw against Lorient due to recovery from a viral infection.
7. Neymar joined PSG from Barcelona for a world-record fee of 222 million euros ($244 million) six years ago.
8. PSG received a world-record $332 million bid from Al-Hilal for Kylian Mbappe.
9. Mbappe is in the last year of his contract and has been involved in a transfer standoff.
10. Mbappe has been strongly linked with a move to Real Madrid next season on a free transfer.
11. Mbappe returned to training following “constructive and positive talks” with PSG.
12. Marco Verratti was not selected by new coach L